In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pdf2image pytesseract PyPDF2 pillow
!apt-get install -y tesseract-ocr
!sudo apt update && sudo apt install -y tesseract-ocr
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Adjust if needed
!apt-get install -y poppler-utils

In [ ]:
import shutil
import os

# Set the path to the working directory
working_directory = '/kaggle/working/'

# Remove all files in the working directory
for filename in os.listdir(working_directory):
    file_path = os.path.join(working_directory, filename)
    try:
        if os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directories
        else:
            os.remove(file_path)  # Remove files
    except Exception as e:
        print(f"Error removing {file_path}: {e}")


In [ ]:
!pip uninstall -y frontend fitz
!pip install PyMuPDF

In [ ]:
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
from pytesseract import Output
from PIL import Image
import os

def pdf_to_searchable(input_pdf, output_pdf):
    # Convert PDF pages to images
    images = convert_from_path(input_pdf)

    # Create a new PDF document
    doc = fitz.open()

    for i, image in enumerate(images):
        # Perform OCR on the image
        ocr_result = pytesseract.image_to_pdf_or_hocr(image, extension='pdf')

        # Save the OCR result as a temporary PDF
        temp_pdf_path = f"temp_page_{i}.pdf"
        with open(temp_pdf_path, "wb") as f:
            f.write(ocr_result)

        # Open the temporary PDF and insert it into the main document
        temp_doc = fitz.open(temp_pdf_path)
        doc.insert_pdf(temp_doc)
        temp_doc.close()

        # Remove the temporary PDF
        os.remove(temp_pdf_path)

    # Save the final searchable PDF
    doc.save(output_pdf)
    doc.close()

# Example usage
input_pdf = "/kaggle/input/shakespeare/Tales from Shakespeare.pdf"
output_pdf = "/kaggle/working/searchable_document.pdf"
pdf_to_searchable(input_pdf, output_pdf)
print(f"Searchable PDF saved as {output_pdf}")